In [1]:
import re
import torch
import subprocess
import pandas as pd
import os

#### Check GPU availability

In [2]:
if "CUDA_VISIBLE_DEVICES" not in os.environ:
    os.environ["CUDA_VISIBLE_DEVICES"] = "0"

if torch.cuda.is_available():
    device = torch.device("cuda")
    gpu_info = os.popen('nvidia-smi --query-gpu=utilization.gpu --format=csv,noheader,nounits').readlines()
    gpu_available = 100 - int(gpu_info[0].strip())
    print(f"\033[1m\033[34mGPU availability in own1: \033[91m{gpu_available}%\033[0m")
else:
    device = torch.device("cpu")
    print('GPU is not available, using CPU instead')

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

GPU availability in own1: 1%


## Pass the Protein and Ligands directory 

In [3]:
protein_path = "aganitha_data/kif18a.pdbqt"
ligands_dir    = "aganitha_data/bbio_mols_pdbqt"

output_file_path = 'aganitha_data/output.txt'
with open(output_file_path, 'w') as output_file:
    pass

## Execute Vina_GPU_2.0 Command

In [4]:
command = (
    f"../Vina-GPU "
    f"--receptor {protein_path} "
    f"--ligand_directory {ligands_dir} "
    f"--center_x 130.64 "
    f"--center_y 115.49 "
    f"--center_z 131.85 "
    f"--size_x 23.71 "
    f"--size_y 23.22 "
    f"--size_z 22.74 "
    f"--thread 8000 "
    f"> {output_file_path}"
)

subprocess.run(command, shell=True)

CompletedProcess(args='../Vina-GPU --receptor aganitha_data/kif18a.pdbqt --ligand_directory aganitha_data/bbio_mols_pdbqt --center_x 130.64 --center_y 115.49 --center_z 131.85 --size_x 23.71 --size_y 23.22 --size_z 22.74 --thread 8000 > aganitha_data/output.txt', returncode=0)

## Extract Rank 1 affinity values from the output

In [5]:
with open(output_file_path, "r") as file:
    output = file.read()

pattern = r'Refining ligand (.+?) results.*?\n\s*1\s+([-+]?\d*\.\d+)\s+\d+\.\d+\s+\d+\.\d+'
matches = re.findall(pattern, output, re.DOTALL)
matches = [(os.path.basename(name), float(affinity)) for name, affinity in matches]
df = pd.DataFrame(matches, columns=['Name', 'Affinity (kcal/mol)'])
df['Affinity (kcal/mol)'] = pd.to_numeric(df['Affinity (kcal/mol)'], errors='coerce')
df = df.sort_values(by='Affinity (kcal/mol)', ascending=True).reset_index(drop=True)

display(df)
print(df.shape)

,Name,Affinity (kcal/mol)
0,AGAN245-df-11-CN,-14.0
1,6,-13.4
2,AGAN 245-df-2-CN,-13.2
3,AGAN245-df-15-CN,-12.7
4,BB_43,-12.6
5,BB_36,-12.4
6,BB_9,-12.2
7,BB_39,-12.0
8,BB_41,-11.9
9,BB_42,-11.8


(23, 2)


.

**Runtime for Docking 23 Molecules:**
- AutoDock-Vina-**CPU** total runtime = 688.8 s
- AutoDock-Vina-**GPU** total runtime = 64.3 s